In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import os


In [ ]:
class LinearReg:
    def __init__(self, alpha=0.1):
        self.weights = None
        self.bias = None
        self.alpha = alpha

    def fit(self, X, y):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        n_samples, n_features = X_b.shape
        identity_matrix = np.eye(n_features)
        identity_matrix[0, 0] = 0

        theta = (
            np.linalg.inv(X_b.T.dot(X_b) + self.alpha * identity_matrix)
            .dot(X_b.T)
            .dot(y)
        )
        self.bias = theta[0]
        self.weights = theta[1:]

    def predict(self, X):
        return X.dot(self.weights) + self.bias

In [ ]:
def create_interaction_features(X):
    """Create interaction features between pairs of features, with improved memory efficiency"""
    feature_names = X.columns
    interactions = pd.DataFrame()

    for i, col1 in enumerate(feature_names):
        for j, col2 in enumerate(feature_names[i+1:], i+1):
            interactions[f"{col1}_{col2}"] = X[col1] * X[col2]

    return pd.concat([X, interactions], axis=1)

def remove_outliers(X, y, threshold=3):
    """Remove outliers using IQR method instead of z-score"""
    Q1 = X.quantile(0.25)
    Q3 = X.quantile(0.75)
    IQR = Q3 - Q1
    outlier_mask = ~((X < (Q1 - threshold * IQR)) | (X > (Q3 + threshold * IQR))).any(axis=1)
    return X[outlier_mask].reset_index(drop=True), y[outlier_mask].reset_index(drop=True)

In [ ]:
#Define Evaluation Metric Functions
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def r_squared(y_true, y_pred):
    y_mean = np.mean(y_true)
    total_variance = np.sum((y_true - y_mean) ** 2)
    explained_variance = np.sum((y_true - y_pred) ** 2)
    return 1 - (explained_variance / total_variance)

In [ ]:
file_path = "./dataset/diabetes.csv"
if not os.path.exists(file_path):
    print('downloading Dataset...')
    iris = load_diabetes()
    store_iris = pd.DataFrame(
        data=np.c_[iris["data"], iris["target"]],
        columns=iris["feature_names"] + ["target"],
    ).to_csv(file_path,index=None)



df = pd.read_csv(file_path)

df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
 10  target  442 non-null    float64
dtypes: float64(11)
memory usage: 38.1 KB


In [ ]:
df.isnull().sum()

,0
age,0
sex,0
bmi,0
bp,0
s1,0
s2,0
s3,0
s4,0
s5,0
s6,0


In [ ]:
X = df.drop(['target'],axis = 1)
y = df['target']
print("Original shape:", X.shape)
X_clean, y_clean = remove_outliers(X, y, threshold=2.5)
print("Shape after removing outliers:", X_clean.shape)

#split dataset
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

Original shape: (442, 10)
Shape after removing outliers: (441, 10)


In [ ]:
# 3. Create interaction features
X_train_interactions = create_interaction_features(X_train)
X_test_interactions = create_interaction_features(X_test)

# Add polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_interactions)
X_test_poly = poly.transform(X_test_interactions)

#feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

# 4. Feature selection to reduce dimensionality
k_best = SelectKBest(score_func=mutual_info_regression, k='all')
X_train_selected = k_best.fit_transform(X_train_interactions, y_train)
X_test_selected = k_best.transform(X_test_interactions)

# Get selected feature masks
selected_features_mask = k_best.get_support()
selected_feature_names = X_train_interactions.columns[selected_features_mask].tolist()

# Convert back to DataFrame with selected features
X_train_selected = pd.DataFrame(X_train_selected, columns=selected_feature_names)
X_test_selected = pd.DataFrame(X_test_selected, columns=selected_feature_names)

In [ ]:
models = {
    'Lasso': LassoCV(
        cv=5,
        random_state=42,
        max_iter=10000,
        tol=1e-3,
        eps=1e-3,
        n_alphas=100,
        selection='random'  # Randomized coordinate descent
    ),
    'Ridge': RidgeCV(
        cv=5,
        alphas=np.logspace(-3, 3, 50)  # Reduced range and number of alphas
    ),
    'ElasticNet': ElasticNetCV(
        cv=5,
        random_state=42,
        max_iter=10000,
        tol=1e-3,
        eps=1e-3,
        n_alphas=100,
        l1_ratio=[.1, .5, .9],  # Reduced number of l1_ratio values
        selection='random'  # Randomized coordinate descent
    )
}

In [ ]:
results = {}
feature_importances = {}

for name, model in models.items():
    print(f"\nTraining {name}...")

    # Create pipeline
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])

    # Fit pipeline
    pipeline.fit(X_train_selected, y_train)

    # Make predictions
    y_pred = pipeline.predict(X_test_selected)

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Cross-validation score
    cv_scores = cross_val_score(pipeline, X_train_selected, y_train, cv=5, scoring='r2')

    results[name] = {
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'CV R² (mean)': cv_scores.mean(),
        'CV R² (std)': cv_scores.std()
    }

    # Store feature importance
    if hasattr(pipeline.named_steps['model'], 'coef_'):
        feature_importances[name] = pd.DataFrame({
            'Feature': selected_feature_names,
            'Coefficient': np.abs(pipeline.named_steps['model'].coef_)
        }).sort_values('Coefficient', ascending=False)


Training Lasso...

Training Ridge...

Training ElasticNet...


In [ ]:
print("\nModel Comparison:")
for name, metrics in results.items():
    print(f"\n{name} Results:")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.3f}")

    # Print best alpha for models that have it
    if hasattr(models[name], 'alpha_'):
        print(f"Best alpha: {models[name].alpha_:.6f}")

    # Print best l1_ratio for ElasticNet
    if name == 'ElasticNet' and hasattr(models[name], 'l1_ratio_'):
        print(f"Best l1_ratio: {models[name].l1_ratio_:.3f}")


Model Comparison:

Lasso Results:
RMSE: 56.842
MAE: 46.216
R²: 0.491
CV R² (mean): 0.482
CV R² (std): 0.070
Best alpha: 3.697407

Ridge Results:
RMSE: 56.914
MAE: 46.378
R²: 0.490
CV R² (mean): 0.467
CV R² (std): 0.079
Best alpha: 79.060432

ElasticNet Results:
RMSE: 56.931
MAE: 46.261
R²: 0.489
CV R² (mean): 0.477
CV R² (std): 0.071
Best alpha: 1.658500
Best l1_ratio: 0.900


In [ ]:
# Print feature importance for the best model
best_model_name = max(results, key=lambda k: results[k]['CV R² (mean)'])
print(f"\nTop 10 Most Important Features ({best_model_name}):")
print(feature_importances[best_model_name].head(10))


Top 10 Most Important Features (Lasso):
    Feature  Coefficient
2       bmi    23.937962
8        s5    21.403052
3        bp    11.523293
6        s3     8.155120
10  age_sex     5.115689
1       sex     3.564557
27   bmi_bp     2.816910
53    s4_s6     2.098413
33   bmi_s6     1.302496
18   age_s6     1.114210
